In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Iceberg + Nessie") \
    .config('spark.jars.packages',
            'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.0,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.103.3,org.apache.iceberg:iceberg-nessie:1.9.0') \
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog") \
    .config("spark.sql.catalog.nessie.uri", "http://localhost:19120/api/v1") \
    .config("spark.sql.catalog.nessie.ref", "main") \
    .config("spark.sql.catalog.nessie.warehouse", "hdfs://172.28.0.11:8020/warehouse") \
    .getOrCreate()
spark

25/06/14 21:16:50 WARN Utils: Your hostname, cj-OptiPlex-3020 resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
25/06/14 21:16:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/cj/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cj/.ivy2/cache
The jars for the packages stored in: /home/cj/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-nessie added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9d6fc64-61e6-4668-a28e-a13659aeea63;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.103.3 in central
	found org.apache.iceberg#iceberg-nessie;1.9.0 in central
	found org.apache.iceberg#iceberg-api;1.9.0 in central
	found org.slf4j#slf4j-api;2.0.17 in central
	found org.apache.iceberg#iceberg-bundled-guava;1.9.0 in central
	found org.apache.iceberg#iceberg-common;1.9.0 in central
	found org.apache.iceberg#iceberg-core;1.9.0 in central
	found org.apache.avro#avro;1.12.0 in cent

In [ ]:
# # Garante que o namespace exista
# spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.data")
# spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.new_data")
# spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.batch")
# spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.speed")

In [12]:
print('"nessie.new_data"')
spark.sql("SHOW TABLES IN nessie.new_data").show(truncate=False)

"nessie.new_data"
+---------+---------------------------+-----------+
|namespace|tableName                  |isTemporary|
+---------+---------------------------+-----------+
|new_data |clientes_20250614T182256   |false      |
|new_data |itens_venda_20250614T183644|false      |
|new_data |itens_venda_20250614T190255|false      |
|new_data |itens_venda_20250614T190335|false      |
|new_data |itens_venda_20250614T212103|false      |
|new_data |new_data_map               |false      |
|new_data |produtos_20250614T182256   |false      |
|new_data |vendas_20250614T183639     |false      |
|new_data |vendas_20250614T190250     |false      |
|new_data |vendas_20250614T190330     |false      |
|new_data |vendas_20250614T212058     |false      |
|new_data |vendedores_20250614T182256 |false      |
+---------+---------------------------+-----------+



In [13]:
faturamento_diario_df=spark.read.table('nessie.new_data.new_data_map')
faturamento_diario_df.show(truncate=False)

+-----------+-------------------+-------------------------------------------+
|table      |date               |path                                       |
+-----------+-------------------+-------------------------------------------+
|vendedores |2025-06-14 18:22:56|nessie.new_data.vendedores_20250614T182256 |
|itens_venda|2025-06-14 19:03:35|nessie.new_data.itens_venda_20250614T190335|
|vendas     |2025-06-14 18:36:39|nessie.new_data.vendas_20250614T183639     |
|vendas     |2025-06-14 21:20:58|nessie.new_data.vendas_20250614T212058     |
|itens_venda|2025-06-14 19:02:55|nessie.new_data.itens_venda_20250614T190255|
|vendas     |2025-06-14 19:03:30|nessie.new_data.vendas_20250614T190330     |
|produtos   |2025-06-14 18:22:56|nessie.new_data.produtos_20250614T182256   |
|itens_venda|2025-06-14 18:36:44|nessie.new_data.itens_venda_20250614T183644|
|itens_venda|2025-06-14 21:21:03|nessie.new_data.itens_venda_20250614T212103|
|clientes   |2025-06-14 18:22:56|nessie.new_data.clientes_202506

In [6]:
print('"nessie.data"')
spark.sql("SHOW TABLES IN nessie.data").show(truncate=False)

"nessie.data"
+---------+-----------+-----------+
|namespace|tableName  |isTemporary|
+---------+-----------+-----------+
|data     |clientes   |false      |
|data     |itens_venda|false      |
|data     |produtos   |false      |
|data     |vendas     |false      |
|data     |vendedores |false      |
+---------+-----------+-----------+



In [7]:

faturamento_diario_df=spark.read.table('nessie.data.vendas')
faturamento_diario_df.show(truncate=False)

+--------+-----------+----------+----------+--------+-------------------+
|id_venda|id_vendedor|id_cliente|data      |total   |criacao            |
+--------+-----------+----------+----------+--------+-------------------+
|1       |8          |247       |2025-06-14|1128.00 |2025-06-14 18:23:00|
|2       |1          |30        |2025-06-14|36804.00|2025-06-14 18:23:05|
|3       |9          |125       |2025-06-14|1128.00 |2025-06-14 18:23:10|
|4       |4          |96        |2025-06-14|27603.00|2025-06-14 18:23:15|
|5       |9          |205       |2025-06-14|61964.00|2025-06-14 18:23:20|
|6       |3          |62        |2025-06-14|2966.00 |2025-06-14 18:23:25|
|7       |7          |139       |2025-06-14|7658.00 |2025-06-14 18:23:30|
|8       |3          |216       |2025-06-14|564.00  |2025-06-14 18:23:35|
|9       |3          |54        |2025-06-14|920.00  |2025-06-14 18:23:40|
|10      |8          |60        |2025-06-14|2966.00 |2025-06-14 18:23:45|
|11      |4          |64        |2025-

In [8]:
print('"nessie.batch"')
spark.sql("SHOW TABLES IN nessie.batch").show(truncate=False)

"nessie.batch"
+---------+--------------------+-----------+
|namespace|tableName           |isTemporary|
+---------+--------------------+-----------+
|batch    |faturamento_diario  |false      |
|batch    |ticket_medio_cliente|false      |
|batch    |vendas_por_vendedor |false      |
+---------+--------------------+-----------+



In [9]:
ticket_medio_cliente_df=spark.read.table('nessie.batch.faturamento_diario')
ticket_medio_cliente_df.show(truncate=True)
ticket_medio_cliente_df.count()

+----------+--------+----------+--------------+------------+
|      data|maior_id|qtd_vendas|total_faturado|ticket_medio|
+----------+--------+----------+--------------+------------+
|2025-06-14|     474|       474|    8460282.00|    17848.70|
+----------+--------+----------+--------------+------------+



1

In [10]:
ticket_medio_cliente_df=spark.read.table('nessie.batch.ticket_medio_cliente')
ticket_medio_cliente_df.show(truncate=True)
ticket_medio_cliente_df.count()

+--------------------+--------+----------+-----------+------------+
|     hash_id_cliente|maior_id|qtd_vendas|total_gasto|ticket_medio|
+--------------------+--------+----------+-----------+------------+
|011af72a910ac4acf...|     406|         2|   23284.00|    11642.00|
|01d54579da446ae1e...|     390|         1|     575.00|      575.00|
|02d20bbd7e394ad59...|     306|         1|     690.00|      690.00|
|031b4af5197ec30a9...|     471|         3|   53695.00|    17898.33|
|05ada863a4cf9660f...|     337|         3|    1498.00|      499.33|
|08490295488a11890...|     474|         1|   11820.00|    11820.00|
|0a2d643bfd24a028c...|     331|         3|    1369.00|      456.33|
|0b918943df0962bc7...|     463|         5|   84934.00|    16986.80|
|0e17daca5f3e175f4...|     136|         2|   65470.00|    32735.00|
|0f4121d0ef1df4c86...|     380|         4|   87757.00|    21939.25|
|0f8ef3377b30fc47f...|     131|         4|  194349.00|    48587.25|
|0fd42b3f73c448b34...|     183|         2|     9

206

In [ ]:
print('"nessie.speed"')
spark.sql("SHOW TABLES IN nessie.speed").show(truncate=False)

In [8]:
faturamento_diario_df=spark.read.table('nessie.speed.faturamento_diario')
faturamento_diario_df.show(truncate=False)
faturamento_diario_df.count()

+-------------------+-------------------+----------+--------+----------+--------------+------------+
|janela_inicio      |janela_fim         |data      |maior_id|qtd_vendas|total_faturado|ticket_medio|
+-------------------+-------------------+----------+--------+----------+--------------+------------+
|2025-06-13 06:52:10|2025-06-13 06:52:20|2025-06-13|5587    |1         |930.0         |930.0       |
|2025-06-13 08:06:00|2025-06-13 08:06:10|2025-06-13|5638    |2         |16556.0       |8278.0      |
|2025-06-13 08:04:40|2025-06-13 08:04:50|2025-06-13|5622    |1         |17704.0       |17704.0     |
|2025-06-13 08:06:10|2025-06-13 08:06:20|2025-06-13|5640    |2         |63204.0       |31602.0     |
|2025-06-13 08:05:20|2025-06-13 08:05:30|2025-06-13|5630    |2         |12750.0       |6375.0      |
|2025-06-13 08:05:10|2025-06-13 08:05:20|2025-06-13|5628    |2         |18105.0       |9052.5      |
|2025-06-13 08:04:50|2025-06-13 08:05:00|2025-06-13|5624    |2         |64637.0       |3231

15

In [ ]:
ticket_medio_cliente_df=spark.read.table('nessie.speed.ticket_medio_cliente')
ticket_medio_cliente_df.show(truncate=False)
ticket_medio_cliente_df.count()

In [ ]:
vendas_por_vendedor_df=spark.read.table('nessie.speed.vendas_por_vendedor')
vendas_por_vendedor_df.show(truncate=False)
vendas_por_vendedor_df.count()

In [ ]:
# spark.sql("DROP TABLE IF EXISTS nessie.dwh.homolog")

spark.sql('DROP TABLE IF EXISTS nessie.speed.vendas_por_vendedor')

In [ ]:
spark.sql("DROP NAMESPACE IF EXISTS nessie.new_data CASCADE")

In [ ]:
spark.stop()